# ライブラリ

In [1]:
import numpy as np
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit.circuit import Gate
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import QasmSimulator

# Swap Test アルゴリズム

### 正規化された2つの配列の準備

In [2]:
N = 7
vec_a = np.random.rand(N) * 2 - 1
vec_a /= np.linalg.norm(vec_a)
vec_b = np.random.rand(N) * 2 - 1
vec_b /= np.linalg.norm(vec_b)

### 用意された配列の確認

In [3]:
print(vec_a)
print(vec_b)
inner_product = np.dot(vec_a, vec_b)
print(inner_product)

[-0.3137755   0.02689877  0.39371444  0.37934842 -0.54859149  0.53702283
  0.1120668 ]
[ 0.42808987  0.22280151 -0.14731029 -0.68897765 -0.38754682 -0.25751632
 -0.23281072]
-0.3994694047489489


### グラムシュミットの正規直交化法

In [4]:
def ceil_log2(N: int) -> int:
    retval = 0
    tmp = 1
    while tmp < N:
        tmp *= 2
        retval += 1
    return retval


def make_unitary_matrix(N: int, vec: np.ndarray) -> np.ndarray:
    n = ceil_log2(N)
    retmat = np.zeros((2**n, 2**n))
    retmat[:N, 0] = vec
    for i in range(1, N):
        tmp = np.zeros(N)
        tmp[i] = 1
        for j in range(i):
            tmp -= np.dot(tmp, retmat[:N, j]) * retmat[:N, j]
        retmat[:N, i] = tmp / np.linalg.norm(tmp)
    for i in range(N, 2**n):
        retmat[i, i] = 1
    return retmat

### 振幅符号化の実装

In [5]:
def amplitude_coding(N: int, vec: np.ndarray, label: str = "") -> Gate:
    n = ceil_log2(N)
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr, name=f"encoding{label}")
    arr = make_unitary_matrix(N, vec)
    qc.append(UnitaryGate(arr), qr)
    return qc.to_gate(label=f"encoding{label}")

### Swap Test アルゴリズムの実装

In [6]:
def controlled_swap_gate(n) -> Gate:
    qr_ancillary = QuantumRegister(1)
    qr_a = QuantumRegister(n)
    qr_b = QuantumRegister(n)
    qc = QuantumCircuit(qr_ancillary, qr_a, qr_b)
    for i in range(n):
        qc.ccx(qr_ancillary, qr_a[i], qr_b[i])
        qc.ccx(qr_ancillary, qr_b[i], qr_a[i])
        qc.ccx(qr_ancillary, qr_a[i], qr_b[i])
    return qc.to_gate(label="swap")


def swap_test(N: int, vec_a: np.ndarray, vec_b: np.ndarray) -> QuantumCircuit:
    n = ceil_log2(N)
    qr_ancillary = QuantumRegister(1, name="ancillary")
    qr_a = QuantumRegister(n, name="data_a")
    qr_b = QuantumRegister(n, name="data_b")
    cr = ClassicalRegister(1, name="measure")
    qc = QuantumCircuit(qr_ancillary, qr_a, qr_b, cr)

    qc.append(amplitude_coding(N, vec_a, " a"), qr_a)
    qc.append(amplitude_coding(N, vec_b, " b"), qr_b)
    qc.h(qr_ancillary)
    qc.append(controlled_swap_gate(n), qr_ancillary[:]+qr_a[:]+qr_b[:])
    qc.h(qr_ancillary)
    
    qc.measure(qr_ancillary, cr)
    return qc

### Swap Test アルゴリズムの実行結果処理

In [7]:
def inner_product_from_result(result: dict) -> float:
    count = result["0"] + result["1"]
    p = result["0"] / count
    return 2*p - 1

### 回路図

In [8]:
qc_swap_test = swap_test(N, vec_a, vec_b)
qc_swap_test.draw()

┌───┐     ┌───────┐┌───┐┌─┐
ancillary: ─────┤ H ├─────┤0      ├┤ H ├┤M├
           ┌────┴───┴────┐│       │└───┘└╥┘
 data_a_0: ┤0            ├┤1      ├──────╫─
           │             ││       │      ║ 
 data_a_1: ┤1 encoding a ├┤2      ├──────╫─
           │             ││       │      ║ 
 data_a_2: ┤2            ├┤3 swap ├──────╫─
           ├─────────────┤│       │      ║ 
 data_b_0: ┤0            ├┤4      ├──────╫─
           │             ││       │      ║ 
 data_b_1: ┤1 encoding b ├┤5      ├──────╫─
           │             ││       │      ║ 
 data_b_2: ┤2            ├┤6      ├──────╫─
           └─────────────┘└───────┘      ║ 
measure: 1/══════════════════════════════╩═
                                         0

### 実行

In [9]:
backend = QasmSimulator()
qc_compiled = transpile(qc_swap_test, backend)
job_sim = backend.run(qc_compiled, shots=8192)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_compiled)
swap_test_ip = inner_product_from_result(counts)
print(f"true |inner product|^2: {inner_product**2:.4f}")
print(f"calc |inner product|^2: {swap_test_ip:.4f}")

true |inner product|^2: 0.1596
calc |inner product|^2: 0.1570


# 改良後 Swap Test アルゴリズム

### 改良後 Swap Test アルゴリズムの実装

In [10]:
def swap_test_2(N: int, vec_a: np.ndarray, vec_b: np.ndarray) -> QuantumCircuit:
    n = ceil_log2(N)
    qr_ancillary = QuantumRegister(1, name="ancillary")
    qr = QuantumRegister(n, name="data")
    cr = ClassicalRegister(1, name="measure")
    qc = QuantumCircuit(qr_ancillary, qr, cr)

    qc.h(qr_ancillary)
    qc.append(amplitude_coding(N, vec_a, " a").control(1), qr_ancillary[:]+qr[:])
    qc.x(qr_ancillary)
    qc.append(amplitude_coding(N, vec_b, " b").control(1), qr_ancillary[:]+qr[:])
    qc.h(qr_ancillary)
    
    qc.measure(qr_ancillary, cr)
    return qc

### 回路図

In [11]:
qc_swap_test_2 = swap_test_2(N, vec_a, vec_b)
qc_swap_test_2.draw()

┌───┐               ┌───┐               ┌───┐┌─┐
ancillary: ┤ H ├───────■───────┤ X ├───────■───────┤ H ├┤M├
           └───┘┌──────┴──────┐└───┘┌──────┴──────┐└───┘└╥┘
   data_0: ─────┤0            ├─────┤0            ├──────╫─
                │             │     │             │      ║ 
   data_1: ─────┤1 encoding a ├─────┤1 encoding b ├──────╫─
                │             │     │             │      ║ 
   data_2: ─────┤2            ├─────┤2            ├──────╫─
                └─────────────┘     └─────────────┘      ║ 
measure: 1/══════════════════════════════════════════════╩═
                                                         0

### 実行

In [12]:
backend = QasmSimulator()
qc_compiled = transpile(qc_swap_test_2, backend)
job_sim = backend.run(qc_compiled, shots=8192)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_compiled)
swap_test_ip_2 = inner_product_from_result(counts)
print(f"true inner product: {inner_product:.4f}")
print(f"calc inner product: {swap_test_ip_2:.4f}")

true inner product: -0.3995
calc inner product: -0.4033
